In [ ]:
import pandas as pd
from pandas.plotting import table
import pandas_profiling
import matplotlib
matplotlib.style.use('seaborn-darkgrid')
#matplotlib.style.available
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.display import display, Markdown, HTML
from ipywidgets import widgets
from utils import human_size, selectdata, run_all_below, ExplorersSelection

In [ ]:
explorers=selectdata("/tmp/datavmware")
f = ExplorersSelection('RVTools_tabvInfo.csv', explorers)
f.widget()

In [ ]:
runall = widgets.Button(description="Run all cells below", button_style='success')
runall.on_click(run_all_below)
display(runall)

In [ ]:
VMFILE = f.selected_explorers
dataraw = pd.read_csv(VMFILE, encoding="ISO-8859-1")
dataraw.columns

In [ ]:
#pandas_profiling.ProfileReport(dataraw)

### Get VMs (Not templates) not running

In [ ]:
usage = dataraw[['VM', 'Template', 'Guest state', 'Provisioned MB', 'In Use MB', 'Disks']]

In [ ]:
usagefilter1 = usage[usage['Guest state'] == "notRunning"]
usagefilter2 = usagefilter1[usagefilter1['Template'] == False].copy()
usagetotal = usagefilter2.copy()
usagefilter2['Provisioned MB'] = usagefilter2['Provisioned MB'].map(human_size)
usagefilter2['In Use MB'] = usagefilter2['In Use MB'].map(human_size)
usagefilter2

### Total space used by Not Running VMs

In [ ]:
usagetotal[['Provisioned MB', 'In Use MB']].sum().apply(human_size)

In [ ]:
display(Markdown("#### Total VMs not running: {}".format(usagetotal['VM'].count())))

In [ ]:
usagetotal[['VM', 'Disks']]

### Get VM by OS

In [ ]:
vmos = dataraw[['VM', 'OS according to the configuration file']]
vmos[['VM', 'OS according to the configuration file']].groupby('OS according to the configuration file').count()

In [ ]:
vmlinux = vmos[['VM', 'OS according to the configuration file']]
vmlinux = vmlinux[vmlinux['OS according to the configuration file'].fillna("Unknown").str.contains('Linux')].VM.count()
vmwindows = vmos[['VM', 'OS according to the configuration file']]
vmwindows = vmwindows[vmwindows['OS according to the configuration file'].fillna("Unknown").str.contains('Microsoft')].VM.count()
vmtotal = vmos[['VM', 'OS according to the configuration file']].VM.count()

for ostype in zip([vmlinux, vmwindows, vmtotal], ['VM linux', 'VM Windows', 'VM Total']):
    display(Markdown("#### {}: {}".format(ostype[1], ostype[0])))

In [ ]:
vms = pd.DataFrame([vmlinux, vmwindows], ['VM linux', 'VM Windows'])
vms.columns = ['VMs']
vms.plot.pie(figsize=(7, 7), y=0, legend=False)

In [ ]:
vmosgroups = dataraw[['VM', 'OS according to the configuration file']]
vmosgroups.groupby('OS according to the configuration file').count().plot.pie(figsize=(12, 12),y=0)
plt.legend(loc="best", bbox_to_anchor=(0.0, 0.45))